In [30]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "/usr/local/spark/jars/mongo-spark-connector_2.12-3.0.2.jar,/usr/local/spark/jars/mongo-java-driver-3.12.9.jar" pyspark-shell'

In [31]:
from pyspark.sql import SparkSession
# fix read bug, basically turn off sampling
spark = SparkSession.builder.appName("TwitterMongo") \
.config("spark.mongodb.input.database", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.input.uri", "mongodb://localhost:27017/twitter.tweets") \
.config("spark.mongodb.read.sql.inferSchema.mapTypes.enabled", "FALSE") \
.config("spark.mongodb.output.uri","mongodb://localhost:27017/twitter.tweets").getOrCreate()

In [32]:
#df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [33]:
# create a spark session
spark = SparkSession \
.builder \
.master("local") \
.appName("ABC") \
.config("spark.driver.memory", "15g") \
.config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/twitter") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:2.12-3.0.2') \
.getOrCreate()
# read data from mongodb collection "questions" into a dataframe "df"
df = spark.read \
.format("com.mongodb.spark.sql.DefaultSource") \
.option("uri", "mongodb://localhost:27017/twitter") \
.option("database", "twitter") \
.option("collection", "tweets") \
.load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: void (nullable = true)
 |-- coordinates: void (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |    |-- title: string (

In [34]:
df.show()

2023-05-14 11:45:38,058 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1296.8 KiB


+--------------------+------------+-----------+--------------------+------------------+--------------------+--------------------+--------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-----------+--------------------+-------------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|                 _id|contributors|coordinates|          created_at|display_text_range|            entities|   extended_entities|extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|plac

In [35]:
df.createOrReplaceTempView("tweets")

In [36]:
dfEnTwt = spark.sql("SELECT * FROM tweets WHERE lang = 'en' AND text LIKE '%euro%'")
dfEnTwt.createOrReplaceTempView("en_tweets")
dfEnTwt.show()

2023-05-14 11:45:47,983 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1300.0 KiB


+--------------------+------------+-----------+--------------------+------------------+--------------------+-----------------+--------------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-----------+--------------------+-------------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|                 _id|contributors|coordinates|          created_at|display_text_range|            entities|extended_entities|      extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lan

In [37]:
dfEnTwt.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: void (nullable = true)
 |-- coordinates: void (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |    |-- title: string (

In [38]:
# select tweet id, geo, lang, quoted_status,quoted_status.geo 
# Having a look at some the data
dfOne = spark.sql("SELECT DISTINCT id,  text, quote_count, reply_count, retweet_count, favorite_count, geo, place, lang, \
                  quoted_status,quoted_status.geo, quoted_status.text, user.name, user.location   \
                  FROM en_tweets")

In [39]:
dfOne.show()

+-------------------+--------------------+-----------+-----------+-------------+--------------+----+-----+----+--------------------+----+--------------------+--------------------+-------------------+
|                 id|                text|quote_count|reply_count|retweet_count|favorite_count| geo|place|lang|       quoted_status| geo|                text|                name|           location|
+-------------------+--------------------+-----------+-----------+-------------+--------------+----+-----+----+--------------------+----+--------------------+--------------------+-------------------+
|1444780630374588417|RT @StockholmCF: ...|          0|          0|            0|             0|null| null|  en|                null|null|                null|Sir Boddington MB...|        Switzerland|
|1424665478170136578|@BillyHynes @Scot...|          0|          0|            0|             0|null| null|  en|                null|null|                null|Paul Harrison-Davies|               null|


In [40]:
#pip install plotly

In [41]:
import pyspark.pandas as ps
import plotly
dfDay= spark.sql("SELECT DISTINCT CAST(substring(created_at, 27, 4) AS INT) as Year, \
          CAST(from_unixtime(unix_timestamp(substring(created_at, 5, 3), 'MMM'), 'MM') As INT) as Month, \
          CAST(substring(created_at, 9, 2) AS INT) as Day, \
          CAST(count(id) AS INT) as TweetCount \
          FROM en_tweets \
          WHERE lang = 'en' AND text LIKE '%euro%' GROUP BY substring(created_at, 27, 4), \
          substring(created_at, 5, 3), \
          substring(created_at, 9, 2)")
dfDay.createOrReplaceTempView("tweetsByDay")

dfDay = spark.sql("SELECT * FROM tweetsByDay ORDER BY Year, Month, Day")




In [42]:

dfDay.show()

+----+-----+---+----------+
|Year|Month|Day|TweetCount|
+----+-----+---+----------+
|2021|    8|  9|        20|
|2021|    8| 10|         5|
|2021|   10|  3|        10|
+----+-----+---+----------+



In [43]:
tempdf = ps.DataFrame(dfDay)

tempdf.plot(kind='bar', x='Day', y='TweetCount')

2023-05-14 11:45:58,390 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:45:58,395 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:45:58,624 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:45:58,771 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:45:58,959 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:45:58,966 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

In [44]:
dfWeek = spark.sql("SELECT Year, weekofyear(make_date(Year, Month, Day)) as wkofYr , SUM(TweetCount) as TweetCountbyWeek \
                   FROM tweetsByDay \
                   GROUP BY Year, weekofyear(make_date(Year, Month, Day))")

dfWeek.createOrReplaceTempView("tweetsByWeek")
dfWeek.show()

+----+------+----------------+
|Year|wkofYr|TweetCountbyWeek|
+----+------+----------------+
|2021|    32|              25|
|2021|    39|              10|
+----+------+----------------+



In [45]:
tempdf = ps.DataFrame(dfWeek)

tempdf.plot(kind='bar', x='wkofYr', y='TweetCountbyWeek')

2023-05-14 11:46:00,836 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:00,839 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:01,009 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:01,198 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:01,315 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:01,318 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

In [46]:
# get the tweet count by month
dfMonth = spark.sql("SELECT Year, Month, SUM(TweetCount) as TweetCountbyMonth \
                     FROM tweetsByDay GROUP BY Year, Month")
dfMonth.createOrReplaceTempView("tweetsByMonth")
dfMonth.show()

+----+-----+-----------------+
|Year|Month|TweetCountbyMonth|
+----+-----+-----------------+
|2021|    8|               25|
|2021|   10|               10|
+----+-----+-----------------+



In [47]:
tempdf = ps.DataFrame(dfMonth)

tempdf.plot(kind='bar', x='Month', y='TweetCountbyMonth')

2023-05-14 11:46:03,061 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:03,065 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:03,224 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:03,326 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:03,408 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-05-14 11:46:03,410 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

### Next up is text clean up 

In [48]:
#%pip install wordcloud
#%pip install vadersentiment
## sentiment analysis ref: https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/?ref=lbp
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    print("Overall sentiment dictionary is : ", sentiment_dict)
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
 
    #print("Sentence Overall Rated As", end = " ")
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
 
    elif sentiment_dict['compound'] <= - 0.05 :
       print("Negative")
 
    else :
        print("Neutral")
    
    return sentiment_dict['compound']

In [49]:
dfText = spark.sql("SELECT DISTINCT LOWER(text) AS text FROM en_tweets")
dfText.createOrReplaceTempView("text")
dfText.show()


+--------------------+
|                text|
+--------------------+
|@billyhynes @scot...|
|good morning euro...|
|its okay obi, las...|
|rt @coverttorture...|
|roman reloaded is...|
|imagining winning...|
|@tesla @elonmusk ...|
|@eurosteppinn i’m...|
|@swiftftbridgers ...|
|@talksport if mes...|
|@lauriedunsire i ...|
|bloke was in tear...|
|rt @sevian_frang:...|
|@blakemurphyodc w...|
|@koolnerdgang @ka...|
|@logo_daedalus yo...|
|@euro2020 toats e...|
|rt @stockholmcf: ...|
|my euro step to t...|
|#darceyandstacey ...|
+--------------------+
only showing top 20 rows



In [50]:
from pyspark.sql.functions import * 
from pyspark.sql.types import StringType, ArrayType
pattern = r'[^a-zA-Z0-9\s]'

dfText = spark.sql("SELECT * FROM text")
dfText = dfText.withColumn("text", regexp_replace('text', pattern, ' '))
dfText.take(5)

[Row(text=' billyhynes  scott1gray  russell m olson it is  i got hold of the euro format gn marvel did too   another  why did  https   t co bjyvuojdxa'),
 Row(text='good morning euro beans  '),
 Row(text='its okay obi  last year you had no idea what a euro step was and now we re trying'),
 Row(text='rt  coverttorture  boarding a train in france  they are scanning for  vaccinepass if it s not valid you will get a 135 euro ticket '),
 Row(text='roman reloaded is euro dance and hyper pop which was very popular in the 2010s  a completely different sound from p  https   t co rt8pxqfjqh')]

In [51]:
dfText.createOrReplaceTempView("text") # this is cleaned txt
dfText = spark.sql("SELECT TRIM(text) AS text FROM text")

dfText.take(5)

[Row(text='billyhynes  scott1gray  russell m olson it is  i got hold of the euro format gn marvel did too   another  why did  https   t co bjyvuojdxa'),
 Row(text='good morning euro beans'),
 Row(text='its okay obi  last year you had no idea what a euro step was and now we re trying'),
 Row(text='rt  coverttorture  boarding a train in france  they are scanning for  vaccinepass if it s not valid you will get a 135 euro ticket'),
 Row(text='roman reloaded is euro dance and hyper pop which was very popular in the 2010s  a completely different sound from p  https   t co rt8pxqfjqh')]

In [52]:
#now build a word cloud
from wordcloud import WordCloud, STOPWORDS
dfText.createOrReplaceTempView("text") # this is the trimmed txt
dfText = spark.sql("SELECT text FROM text")
dfText.show(5)

+--------------------+
|                text|
+--------------------+
|billyhynes  scott...|
|good morning euro...|
|its okay obi  las...|
|rt  coverttorture...|
|roman reloaded is...|
+--------------------+
only showing top 5 rows



### Tokenize and Stem the tweets

In [53]:
#%pip install nltk

In [54]:
# stackoverflow ref: https://stackoverflow.com/questions/53579444

from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from nltk.stem.snowball import SnowballStemmer

In [55]:
# Tokenize text
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
dfText = tokenizer.transform(dfText).select("text","tokens")

dfText.show(5)

+--------------------+--------------------+
|                text|              tokens|
+--------------------+--------------------+
|billyhynes  scott...|[billyhynes, , sc...|
|good morning euro...|[good, morning, e...|
|its okay obi  las...|[its, okay, obi, ...|
|rt  coverttorture...|[rt, , coverttort...|
|roman reloaded is...|[roman, reloaded,...|
+--------------------+--------------------+
only showing top 5 rows



In [56]:
# Remove stopwords
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
dfText = remover.transform(dfText).select("text","filtered")

dfText.show(5)

+--------------------+--------------------+
|                text|            filtered|
+--------------------+--------------------+
|billyhynes  scott...|[billyhynes, , sc...|
|good morning euro...|[good, morning, e...|
|its okay obi  las...|[okay, obi, , las...|
|rt  coverttorture...|[rt, , coverttort...|
|roman reloaded is...|[roman, reloaded,...|
+--------------------+--------------------+
only showing top 5 rows



In [57]:
# stem the words
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
dfText = dfText.withColumn("filtered_stemmed", stemmer_udf("filtered"))
dfText.show(5)

+--------------------+--------------------+--------------------+
|                text|            filtered|    filtered_stemmed|
+--------------------+--------------------+--------------------+
|billyhynes  scott...|[billyhynes, , sc...|[billyhyn, , scot...|
|good morning euro...|[good, morning, e...|[good, morn, euro...|
|its okay obi  las...|[okay, obi, , las...|[okay, obi, , las...|
|rt  coverttorture...|[rt, , coverttort...|[rt, , coverttort...|
|roman reloaded is...|[roman, reloaded,...|[roman, reload, e...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [58]:
# Filter out short words
filterShortWords = udf(lambda row: [x for x in row if len(x) >= 4], ArrayType(StringType()))
dfText = dfText.withColumn("filtered_stemmed", filterShortWords("filtered_stemmed"))

dfText.show(5)


+--------------------+--------------------+--------------------+
|                text|            filtered|    filtered_stemmed|
+--------------------+--------------------+--------------------+
|billyhynes  scott...|[billyhynes, , sc...|[billyhyn, scott1...|
|good morning euro...|[good, morning, e...|[good, morn, euro...|
|its okay obi  las...|[okay, obi, , las...|[okay, last, year...|
|rt  coverttorture...|[rt, , coverttort...|[coverttortur, bo...|
|roman reloaded is...|[roman, reloaded,...|[roman, reload, e...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

